In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import re
import pandas as pd
from bench_lib.evaluation import load_ai_labels, load_human_labels, krippendorf_alpha

human_labels, questions, comment_cols = load_human_labels()
alphas = [
    krippendorf_alpha(human_labels["post_id"], human_labels[q]) for q in questions
]

def has_disagreement(x):
    for q in questions:
        values = x[q].dropna().unique()
        if len(values) > 1:
            try:
                if krippendorf_alpha(x["post_id"], x[q]) < 0.60:
                    return True
            except ValueError:
                continue
    return False

disagreement_subset = human_labels[
    ["post_id", "classification_by"] + questions
].groupby("post_id").filter(has_disagreement)
disagreement_subset.head(3)

human_labels_long, _, _ = load_human_labels(long=True)

models = [
    "google/gemma-3-4b-it",
    "google/gemma-3-12b-it",
    "google/gemma-3-27b-it",
    "Qwen/Qwen2.5-VL-3B-Instruct",
    "Qwen/Qwen2.5-VL-7B-Instruct",
    "Qwen/Qwen2.5-VL-72B-Instruct"
]

run_folders = [
    "toxicainment_videos_log_Temp_0_7_1",
    "toxicainment_videos_log_Temp_0_7_2",
    "toxicainment_videos_log_Temp_0_7_3",
    "toxicainment_videos_log_Temp_0_7_4",
    "toxicainment_videos_log_Temp_0_7_5"
]

base_path = "/home/tanalp/toxicainment/simple_inference_benchmark_results"

model_results = {}

for model in models:
    folders = [
        f"{base_path}/{run_folder}/{model}"
        for run_folder in run_folders
    ]
    ai_labels_long = pd.concat(
        [load_ai_labels([folder], questions, comment_cols) for folder in folders],
        ignore_index=True
    )
    model_results[model] = ai_labels_long

pattern = (
    r"(hashtag(s)?|#\w+|"  # hashtag, hashtags, or #word
    r"[\U0001F600-\U0001F64F]"  # emoticons - Common face emojis (😀-🙏)
    r"|[\U0001F300-\U0001F5FF]"  # symbols & pictographs (includes hearts) (💙, 🇩🇪)
    r"|[\U0001F1E6-\U0001F1FF])"  # regional indicator symbols (used for flags) (💙, 🇩🇪)
)

human_commented = human_labels_long[
    human_labels_long["comment"].notnull() & (human_labels_long["comment"].str.strip() != "")
]

for model_name, ai_labels_long in model_results.items():
    
    common_post_ids = set(human_commented["post_id"]).intersection(set(ai_labels_long["post_id"]))
    
    ai_labels_long_grouped = ai_labels_long.groupby(["post_id", "variable"]).first().reset_index()
    
    human_common_commented = human_commented[human_commented["post_id"].isin(common_post_ids)]
    ai_common_commented = ai_labels_long_grouped[ai_labels_long_grouped["post_id"].isin(common_post_ids)]

    human_common_commented = human_common_commented.assign(
        comment_contains_hashtag=human_common_commented["comment"].str.contains(pattern, regex=True, flags=re.IGNORECASE, na=False),
        comment_contains_audio=human_common_commented["comment"].str.contains("audio|sound|voice|music|song", case=False, na=False)
    )
    ai_common_commented = ai_common_commented.assign(
        comment_contains_hashtag=ai_common_commented["comment"].str.contains(pattern, regex=True, flags=re.IGNORECASE, na=False),
        comment_contains_audio=ai_common_commented["comment"].str.contains("audio|sound|voice|music|song", case=False, na=False)
    )

    human_common_commented = human_common_commented[human_common_commented["post_id"].isin(disagreement_subset["post_id"])]
    ai_common_commented = ai_common_commented[ai_common_commented["post_id"].isin(disagreement_subset["post_id"])]

    merged_comments = pd.merge(
        human_common_commented,
        ai_common_commented,
        on=["post_id", "variable"],
        suffixes=("_human", "_ai"),
        how="inner"
    )
    safe_model_name = model_name.replace("/", "_")
    merged_comments.to_csv(f"merged_comments_{safe_model_name}.csv", index=False)